## Setup

In [12]:
import pandas as pd

# Set path to audiofiles
audios_path = r"Rodrigo\Audios\1 - Campus\1 - SMM00894 (plv)\Data"

# Set path to where spectrograms will be saved
spectrograms_path = r"C:\Users\55149\OneDrive\Desktop\spectrograms"

# Set path to dataframe containing .wav filenames if no such dataframe is available, 
# run the first function "create_wav_dataframe"
df = pd.read_csv(r"C:\Users\55149\OneDrive\Desktop\rain_files.csv",index_col=0,parse_dates=True)

df.head()
df = df.head()

## Create DataFrame of .wav files

In [ ]:
import os
import pandas as pd

def create_wav_dataframe(folder_path):
    """
    Creates a DataFrame with all .wav files found in the given folder path.

    Parameters:
    -----------
    folder_path : str
        The directory path where the .wav audio files are located.
    
    Returns:
    --------
    pandas.DataFrame
        A DataFrame with a single column 'filename' containing the .wav filenames
        found in the specified folder.
    
    Raises:
    -------
    FileNotFoundError:
        If the specified folder path does not exist.
    """
    
    # Check if the folder path exists
    if not os.path.isdir(folder_path):
        raise FileNotFoundError(f"The folder path {folder_path} does not exist.")
    
    # List all .wav files in the folder
    wav_files = [f for f in os.listdir(folder_path) if f.endswith('.wav')]
    
    # Raise a warning if no .wav files are found
    if not wav_files:
        print("Warning: No .wav files found in the specified folder.")
    
    # Create a DataFrame with the .wav filenames
    df = pd.DataFrame(wav_files, columns=['filename'])
    
    return df

In [ ]:
df = create_wav_dataframe(audios_path)
df.head()

## Extract Spectrogram

In [ ]:
import os, librosa
import pandas as pd
import numpy as np
import librosa.display
import matplotlib.pyplot as plt

def extract_spectrogram(df, audios_path, output_folder):
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)
    
    for idx, row in df.iterrows():
        filename = row['filename']
        full_audio_path = os.path.join(audios_path, filename)
        
        # Generate the name for the spectrogram image
        spectrogram_filename = f"{filename[:-4]}.png"
        save_image_path = os.path.join(output_folder, spectrogram_filename)

        # Check if the spectrogram already exists
        if os.path.exists(save_image_path):
            print(f"Spectrogram already exists for {filename}. Skipping...")
            continue

        # Load the audio file and compute the spectrogram
        try:
            y, sr = librosa.load(full_audio_path, sr=None)
            
            # Compute the spectrogram
            D = librosa.amplitude_to_db(librosa.stft(y), ref=np.max)
            
            # Plot the spectrogram without title, ticks, and labels
            plt.figure(figsize=(8, 6))
            librosa.display.specshow(D, sr=sr)
            plt.axis('off')  # Turn off axis
            plt.savefig(save_image_path, bbox_inches='tight', pad_inches=0)
            plt.close()
            print(f"Saved spectrogram for {filename}")

        except FileNotFoundError:
            print(f"Error: The file {full_audio_path} was not found. Skipping to the next file.")
        except Exception as e:
            print(f"An error occurred while processing {filename}: {e}")

In [ ]:
extract_spectrogram(df, audios_path, spectrograms_path)

## Extract Histogram Data

In [14]:
import os
import cv2
import pandas as pd
import numpy as np

def extract_histogram(df, spectrogram_path, filter='bilateral'):
    """
    Computes histograms for spectrogram images and adds them as new columns in the DataFrame.
    If histogram columns already exist in the DataFrame, their values will be overwritten.

    Parameters:
    -----------
    df : pandas.DataFrame
        The DataFrame containing a column with .wav file names.
    spectrogram_path : str
        The directory path where the spectrogram images are located.
    filter : str, optional
        The type of filter to apply to the images before calculating the histogram.
        Options are 'gaussian', 'bilateral', or 'median'. Default is 'bilateral'.

    Returns:
    --------
    pandas.DataFrame
        The original DataFrame with updated columns named 'HIST_1', 'HIST_2', ..., 'HIST_256'
        corresponding to the histogram values of the processed images. Existing histogram columns
        will be overwritten.
    """
    
    # Identify the column that contains .wav filenames
    wav_col = None
    for col in df.columns:
        if df[col].str.endswith('.wav').any():
            wav_col = col
            break

    if wav_col is None:
        raise ValueError("No column with .wav files found in the DataFrame.")

    # Initialize a list to store the histograms
    hist_data = []

    for index, row in df.iterrows():
        # Get the filename from the identified column
        wav_file = row[wav_col]
        filename = wav_file[:-4] + '.png'  # Replace .wav with .png
        image_path = os.path.join(spectrogram_path, filename)

        # Check if the image file exists
        if not os.path.exists(image_path):
            print(f"File {filename} not found, skipping.")
            hist_data.append([np.nan] * 256)
            continue

        # Read the image in grayscale
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            print(f"Unable to read image {filename}, skipping.")
            hist_data.append([np.nan] * 256)
            continue

        # Apply the specified filter
        if filter == 'gaussian':
            filtered_image = cv2.GaussianBlur(image, (5, 5), 0)
        elif filter == 'median':
            filtered_image = cv2.medianBlur(image, 5)
        else:  # Default to bilateral filter
            filtered_image = cv2.bilateralFilter(image, 9, 75, 75)

        # Calculate histogram
        hist = cv2.calcHist([filtered_image], [0], None, [256], [0, 256])

        # Flatten the histogram and append it to hist_data
        hist_data.append(hist.flatten())

    # Create a new DataFrame for the histogram columns
    hist_df = pd.DataFrame(hist_data, columns=[f'HIST_{i}' for i in range(1, 257)], index=df.index)

    # Check for existing histogram columns in the DataFrame and drop them
    existing_hist_cols = [col for col in df.columns if col.startswith('HIST_')]
    if existing_hist_cols:
        df.drop(columns=existing_hist_cols, inplace=True)

    # Concatenate the histogram DataFrame with the original DataFrame
    df = pd.concat([df, hist_df], axis=1)

    return df

In [15]:
df = extract_histogram(df, spectrograms_path, filter='bilateral')
df

File SMM00894_20230214_100000.png not found, skipping.


,filename,rain,rain_class,total_rain,HIST_1,HIST_2,HIST_3,HIST_4,HIST_5,HIST_6,...,HIST_247,HIST_248,HIST_249,HIST_250,HIST_251,HIST_252,HIST_253,HIST_254,HIST_255,HIST_256
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-02-14 10:00:00,SMM00894_20230214_100000.wav,0,no rain,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-02-14 10:05:00,SMM00894_20230214_100500.wav,0,no rain,0.0,52943.0,27975.0,15376.0,10984.0,9316.0,8902.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-02-14 10:10:00,SMM00894_20230214_101011.wav,0,no rain,0.0,161395.0,15507.0,7352.0,5447.0,4463.0,4399.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-02-14 10:15:00,SMM00894_20230214_101500.wav,0,no rain,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-02-14 10:20:00,SMM00894_20230214_102000.wav,0,no rain,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Extract LPC

In [ ]:
import os
import librosa
import pandas as pd
import numpy as np
import warnings
import soundfile as sf

def extract_lpc(df, audio_path, n_components):
    """
    Extracts Linear Predictive Coding (LPC) components from .wav audio files listed in a DataFrame.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        The DataFrame containing a column with .wav file names.
    audio_path : str
        The directory path where the .wav audio files are located.
    n_components : int
        The number of LPC components to extract from each audio file.
    
    Returns:
    --------
    pandas.DataFrame
        The original DataFrame with additional columns named 'LPC_1', 'LPC_2', ..., 'LPC_n'
        corresponding to the extracted LPC components, where 'n' is the value of `n_components`.
        
    Raises:
    -------
    ValueError:
        If no column with .wav files is found in the DataFrame.
        
    Warnings:
    ---------
    UserWarning:
        If no .wav files are found in the identified column, the function will issue a warning
        and return the DataFrame unchanged.
    
    Notes:
    ------
    - This function will attempt to load each .wav file using `librosa`, extract the LPC components,
      and append these components as new columns in the DataFrame.
    - If any audio file fails to load (e.g., missing or corrupted), NaN values will be appended
      for that file's LPC components.
    """
    
    # Identify the column that contains .wav filenames
    wav_col = None
    for col in df.columns:
        if df[col].str.endswith('.wav').any():
            wav_col = col
            break
    
    if wav_col is None:
        raise ValueError("No column with .wav files found")

    # Check if there are no .wav files in the column
    wav_files_found = df[wav_col].str.endswith('.wav').sum()
    if wav_files_found == 0:
        warnings.warn("No .wav files found in the DataFrame.")
        return df  # Return the DataFrame unchanged if no .wav files are found

    # Initialize empty lists to store LPC components
    lpc_cols = {f'LPC_{i+1}': [] for i in range(n_components)}

    # Suppress PySoundFile and audioread warnings
    with warnings.catch_warnings():
        warnings.simplefilter('ignore', category=UserWarning)
        warnings.simplefilter('ignore', category=FutureWarning)

        # Process each audio file and extract LPC components
        for idx, row in df.iterrows():
            wav_file = row[wav_col]
            full_path = os.path.join(audio_path, wav_file)

            try:
                # Load the audio file
                y, sr = librosa.load(full_path, sr=None)

                # Extract LPC coefficients
                lpc = librosa.lpc(y, order=n_components)

                # Fill in the LPC components, only take the first n_components
                for i in range(n_components):
                    lpc_cols[f'LPC_{i+1}'].append(lpc[i] if i < len(lpc) else np.nan)

            except Exception as e:
                # Extract only the filename from the full path
                filename = os.path.basename(full_path)
                # Print a more concise error message
                print(f"Error processing {filename}: {str(e).split(':')[0]}")
                for i in range(n_components):
                    lpc_cols[f'LPC_{i+1}'].append(np.nan)
    
    # Add LPC columns to the DataFrame
    for col, values in lpc_cols.items():
        df[col] = values
    
    return df

In [ ]:
df = extract_lpc(df, audios_path, n_components=10)

## Extract MFCC

In [ ]:
import os
import librosa
import pandas as pd
import numpy as np
import warnings
import soundfile as sf

def extract_mfcc(df, audio_path, n_mfcc):
    """
    Extracts the mean Mel Frequency Cepstral Coefficients (MFCC) components from .wav audio files listed in a DataFrame.

    This function computes the MFCCs for each audio file, retaining only the mean values of each MFCC component,
    which are then added as new columns to the original DataFrame.

    Parameters:
    -----------
    df : pandas.DataFrame
        The DataFrame containing a column with .wav file names.
    audio_path : str
        The directory path where the .wav audio files are located.
    n_mfcc : int
        The number of MFCC components to extract from each audio file.

    Returns:
    --------
    pandas.DataFrame
        The original DataFrame with additional columns named 'MFCC_1', 'MFCC_2', ..., 'MFCC_n'
        corresponding to the mean of the extracted MFCC components, where 'n' is the value of `n_mfcc`.

    Raises:
    -------
    ValueError:
        If no column with .wav files is found in the DataFrame.

    Warnings:
    ---------
    UserWarning:
        If no .wav files are found in the identified column, the function will issue a warning
        and return the DataFrame unchanged.

    Notes:
    ------
    - This function will attempt to load each .wav file using `librosa`, extract the MFCC components,
      and calculate the mean of these components. The mean values are then appended as new columns
      in the DataFrame.
    - If any audio file fails to load (e.g., missing or corrupted), NaN values will be appended
      for that file's MFCC components.
    """
    
    # Identify the column that contains .wav filenames
    wav_col = None
    for col in df.columns:
        if df[col].str.endswith('.wav').any():
            wav_col = col
            break
    
    if wav_col is None:
        raise ValueError("No column with .wav files found")

    # Check if there are no .wav files in the column
    wav_files_found = df[wav_col].str.endswith('.wav').sum()
    if wav_files_found == 0:
        warnings.warn("No .wav files found in the DataFrame.")
        return df  # Return the DataFrame unchanged if no .wav files are found

    # Initialize empty lists to store MFCC mean components
    mfcc_cols = {f'MFCC_{i+1}': [] for i in range(n_mfcc)}

    # Suppress PySoundFile and audioread warnings
    with warnings.catch_warnings():
        warnings.simplefilter('ignore', category=UserWarning)
        warnings.simplefilter('ignore', category=FutureWarning)

        # Process each audio file and extract MFCC components
        for idx, row in df.iterrows():
            wav_file = row[wav_col]
            full_path = os.path.join(audio_path, wav_file)

            try:
                # Load the audio file
                y, sr = librosa.load(full_path, sr=None)

                # Extract MFCC coefficients
                mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
                
                # Compute the mean of the MFCCs along the second axis (time axis)
                mfccs_mean = np.mean(mfccs, axis=1)

                # Fill in the MFCC mean components
                for i in range(n_mfcc):
                    mfcc_cols[f'MFCC_{i+1}'].append(mfccs_mean[i] if i < len(mfccs_mean) else np.nan)

            except Exception as e:
                # Extract only the filename from the full path
                filename = os.path.basename(full_path)
                # Print a more concise error message
                print(f"Error processing {filename}: {str(e).split(':')[0]}")
                for i in range(n_mfcc):
                    mfcc_cols[f'MFCC_{i+1}'].append(np.nan)
    
    # Add MFCC mean columns to the DataFrame
    for col, values in mfcc_cols.items():
        df[col] = values
    
    return df

In [ ]:
df = extract_mfcc(df, audios_path, n_mfcc=10)

## Extract PSD

In [ ]:
import os
import numpy as np
import pandas as pd
import librosa
from scipy.signal import welch
import warnings

# Suppress specific warnings from librosa
warnings.filterwarnings("ignore", category=UserWarning, module='librosa')
warnings.filterwarnings("ignore", category=FutureWarning, module='librosa')

def extract_psd(df, audio_path, nperseg):
    """
    Extracts Power Spectrum Density (PSD) values from .wav files listed in a DataFrame.
    
    Args:
        df (pd.DataFrame): The original DataFrame containing filenames in a column.
        audio_path (str): The path where the .wav audio files are located.
        nperseg (int): Number of samples per segment for Welch's method.
    
    Returns:
        pd.DataFrame: The original DataFrame with additional columns for the PSD values.
    """
    psd_dict = {}
    wav_col = df.select_dtypes(include='object').apply(lambda x: x.str.endswith('.wav')).any(axis=0)
    wav_column = wav_col.index[wav_col].tolist()
    
    if not wav_column:
        print("No .wav files found in the DataFrame.")
        return df

    for idx, row in df.iterrows():
        filename = row[wav_column[0]]
        full_path = os.path.join(audio_path, filename)

        try:
            y, sr = librosa.load(full_path, sr=None)
            freqs, psd = welch(y, fs=sr, nperseg=nperseg)
            rounded_freqs = np.round(freqs).astype(int)
            psd_dict[idx] = {f: psd[i] for i, f in enumerate(rounded_freqs)}

        except Exception as e:
            print(f"Error processing {filename}: {str(e).split(':')[0]}")
            if 'rounded_freqs' in locals():
                psd_dict[idx] = {f: np.nan for f in rounded_freqs}
            else:
                psd_dict[idx] = {f: np.nan for f in range(0, 1)}

    psd_df = pd.DataFrame.from_dict(psd_dict, orient='index')
    df = pd.concat([df, psd_df], axis=1)

    return df

In [ ]:
df = extract_psd(df, audios_path, nperseg=1024)